In [ ]:
**ANALISIS EXPLORATORIO**

#importar librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import triang

#cargar dataset
df = pd.read_csv('/content/drive/MyDrive/rutas_PanamaColon_con_clima_20250914.csv')

#mostrar columnas de duracion min, max y duracion en minutos
cols_check = ["duration_min(minutes)", "duration_max(minutes)", "duration(minutes)"]
print(df[cols_check].head())

#calcular parametros para distribucion triangular
df["min_dur"] = df["duration_min(minutes)"]
df["max_dur"] = df["duration_max(minutes)"]
df["mode_dur"] = df["duration(minutes)"]


#proporcion para la forma (c)
df["c"] = (df["mode_dur"] - df["min_dur"]) / (df["max_dur"] - df["min_dur"])

#distribucion graficada de un viaje
ejemplo = df.iloc[0]
x = np.linspace(ejemplo["min_dur"], ejemplo["max_dur"], 100)
tri_pdf = triang.pdf(x, ejemplo["c"], loc=ejemplo["min_dur"], scale=ejemplo["max_dur"]-ejemplo["min_dur"])

plt.figure(figsize=(8,5))
plt.plot(x, tri_pdf, label=f"Triangular: min={ejemplo['min_dur']}, mode={ejemplo['mode_dur']}, max={ejemplo['max_dur']}")
plt.axvline(ejemplo["mode_dur"], color="r", linestyle="--", label="Duración observada (modo)")
plt.title("Ejemplo de distribución triangular para un viaje")
plt.xlabel("Duración (minutos)")
plt.ylabel("Densidad")
plt.legend()
plt.show()

#relacion clima vs parametros (precipitacion y viento)
fig, axes = plt.subplots(1, 3, figsize=(15,5))

sns.scatterplot(data=df, x="origin_precipitation", y="min_dur", alpha=0.5, ax=axes[0])
axes[0].set_title("Precipitación vs duración mínima")

sns.scatterplot(data=df, x="origin_precipitation", y="mode_dur", alpha=0.5, ax=axes[1])
axes[1].set_title("Precipitación vs duración modo")

sns.scatterplot(data=df, x="origin_precipitation", y="max_dur", alpha=0.5, ax=axes[2])
axes[2].set_title("Precipitación vs duración máxima")

plt.tight_layout()
plt.show()

print(df["wind_level"].value_counts())


print(df.groupby("wind_level")["mode_dur"].median())

plt.figure(figsize=(8,5))
sns.barplot(
    data=df,
    x="wind_level",
    y="mode_dur",
    palette="YlOrRd",
    estimator=np.median,   #
    ci=None                #
)

plt.plot(
    range(len(median_dur)),
    median_dur["mode_dur"],
    marker="o",
    linestyle="-",
    color="black",
    linewidth=2,
    label="Mediana"
)

plt.title("Duración de viajes según nivel de viento (mediana)")
plt.xlabel("Nivel de viento")
plt.ylabel("Duración (minutos)")
plt.legend()
plt.show()


**MODELO TRIANGULAR SIMPLE**

#ejemplo con los parametros globales del dataset
min_dur = df["min_dur"].min()
mode_dur = df["mode_dur"].median()
max_dur = df["max_dur"].max()

#parametros para la distrib. triangular
c = (mode_dur - min_dur) / (max_dur - min_dur)  # forma
loc = min_dur
scale = max_dur - min_dur

#simulacion de viajes con modelo triangular
sim_simple = triang.rvs(c, loc=loc, scale=scale, size=10000)

#grafica
plt.figure(figsize=(8,5))
plt.hist(sim_simple, bins=30, density=True, alpha=0.6, color="skyblue", label="Simulación Triangular")
plt.axvline(mode_dur, color="red", linestyle="--", label=f"Modo observado: {mode_dur}")
plt.title("Distribución Triangular Simple de Duración de Viajes")
plt.xlabel("Duración (minutos)")
plt.ylabel("Densidad")
plt.legend()
plt.show()

**SIMULACION MONTE CARLO CON VARIABLES CLIMATICAS**

#ajusta parametros de la triangular en funciOn del clima
def triangular_params(row):

    min_d = row["min_dur"]
    mode_d = row["mode_dur"]
    max_d = row["max_dur"]

    #ajuste basico segun clima
    if row["origin_precipitation"] > 5:  # lluvia fuerte
        max_d += 10
        mode_d += 5
    if row["origin_windspeed_10m"] > 15:  # viento alto
        max_d += 5

    return min_d, mode_d, max_d

#simulacion Monte Carlo
n_sim = 5000
sim_durations = []

for i in range(n_sim):
    sample = df.sample(1).iloc[0]  # toma una fila aleatoria
    min_d, mode_d, max_d = triangular_params(sample)

    if max_d > min_d:  # solo si hay rango valido
        c = (mode_d - min_d) / (max_d - min_d)
        sim_val = triang.rvs(c, loc=min_d, scale=max_d - min_d, size=1)
        sim_durations.append(sim_val[0])

#grafico
plt.figure(figsize=(8,5))
plt.hist(sim_durations, bins=40, density=True, alpha=0.6, color="orange")
plt.title("Simulación Monte Carlo con Distribución Triangular + Clima")
plt.xlabel("Duración (minutos)")
plt.ylabel("Densidad")
plt.show()

**COMPARACION DE ESCENARIOS CLIMATICOS**

escenarios = {
    "Normal": {"prec": 0, "wind": 5},
    "Lluvia Fuerte": {"prec": 10, "wind": 5},
    "Viento Alto": {"prec": 0, "wind": 20}
}

plt.figure(figsize=(10,6))

for label, clima in escenarios.items():
    sim = []
    for i in range(3000):
        row = df.sample(1).iloc[0]
        min_d, mode_d, max_d = row["min_dur"], row["mode_dur"], row["max_dur"]

        #se aplican ajustes de escenario
        if clima["prec"] > 5:
            max_d += 10
            mode_d += 5
        if clima["wind"] > 15:
            max_d += 5

        if max_d > min_d:
            c = (mode_d - min_d) / (max_d - min_d)
            sim_val = triang.rvs(c, loc=min_d, scale=max_d - min_d, size=1)
            sim.append(sim_val[0])

    sns.kdeplot(sim, label=label)

plt.title("Comparación de escenarios climáticos con Monte Carlo")
plt.xlabel("Duración (minutos)")
plt.ylabel("Densidad")
plt.legend()
plt.show()
